In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Mon Mar 20 03:35:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    52W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Connect google drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**1) Clone the Darknet**

In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15514, done.
remote: Total 15514 (delta 0), reused 0 (delta 0), pack-reused 15514
Receiving objects: 100% (15514/15514), 14.18 MiB | 27.24 MiB/s, done.
Resolving deltas: 100% (10412/10412), done.


**2) Compile Darknet using Nvidia GPU**

In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |             if (iteration_

**3) Configure Darknet network for training YOLO V3**

In [5]:
!cp /content/drive/MyDrive/Traffic/cfg/yolov3_r.cfg cfg/yolov3_r.cfg
!cp /content/drive/MyDrive/Traffic/cfg/obj.data data/obj.data
!cp /content/drive/MyDrive/Traffic/cfg/obj.names data/obj.names


In [6]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2023-03-20 03:37:28--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  40.3MB/s    in 4.3s    

2023-03-20 03:37:32 (36.4 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

In [7]:
!unzip /content/drive/MyDrive/Traffic/images.zip -d data/obj

Streaming output truncated to the last 5000 lines.
  inflating: data/obj/images/20200119_180051_016_jpg.rf.95c7b6c1099818103ea62e6c780e50d6.jpg  
  inflating: data/obj/images/20200119_180051_016_jpg.rf.95c7b6c1099818103ea62e6c780e50d6.txt  
  inflating: data/obj/images/20200119_180051_022_jpg.rf.e2352bdd34ec8ebbf5ef2cad68ee4710.jpg  
  inflating: data/obj/images/20200119_180051_022_jpg.rf.e2352bdd34ec8ebbf5ef2cad68ee4710.txt  
  inflating: data/obj/images/20200119_180051_025_jpg.rf.da30587c937d1c62753c78e0e45589b8.jpg  
  inflating: data/obj/images/20200119_180051_025_jpg.rf.da30587c937d1c62753c78e0e45589b8.txt  
  inflating: data/obj/images/20200119_180051_027_jpg.rf.3787cc91f3bd59d1fe61f69a5b6236f8.jpg  
  inflating: data/obj/images/20200119_180051_027_jpg.rf.3787cc91f3bd59d1fe61f69a5b6236f8.txt  
  inflating: data/obj/images/20200119_180051_030_jpg.rf.2ab83925b9a591092a13de03ac595c4f.jpg  
  inflating: data/obj/images/20200119_180051_030_jpg.rf.2ab83925b9a591092a13de03ac595c4f.txt  

In [8]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/images/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line) #eg:::> 1 0.298818 0.484064 0.105885 0.852590
            numbers = re.findall("[0-9.]+", line)
            print(numbers) #eg:::> ['1', '0.298818', '0.484064', '0.105885', '0.852590']

            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(numbers[0], numbers[1], numbers[2], numbers[3], numbers[4]) #eg:::> 1 0.298818 0.484064 0.105885 0.852590
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Streaming output truncated to the last 5000 lines.
['10', '0.57109375', '0.525', '0.01953125', '0.0296875']
2751 data/obj/images/20200119_135909_010_jpg.rf.45ee71944601d76ce804fbb56702de34.txt
10 0.57109375 0.525 0.01953125 0.0296875
0 0.4796875 0.41875 0.065625 0.30078125
['0', '0.4796875', '0.41875', '0.065625', '0.30078125']
2752 data/obj/images/20200119_184219_016_jpg.rf.7577c68101d227c01ca62a13e84f2dc3.txt
0 0.4796875 0.41875 0.065625 0.30078125
12 0.40390625 0.46875 0.04765625 0.07578125
['12', '0.40390625', '0.46875', '0.04765625', '0.07578125']
2753 data/obj/images/20200119_133232_021_jpg.rf.2badd45ebd6309e897db509b2f1be21c.txt
12 0.40390625 0.46875 0.04765625 0.07578125
0 0.440625 0.40703125 0.05078125 0.1734375
['0', '0.440625', '0.40703125', '0.05078125', '0.1734375']
2754 data/obj/images/20200119_183018_007_jpg.rf.af4f7a80ea75acb342fb61fc0f0e48d2.txt
0 0.440625 0.40703125 0.05078125 0.1734375
12 0.384375 0.38671875 0.08984375 0.15
['12', '0.384375', '0.38671875', '0.0898437

In [9]:
import glob
images_list = glob.glob("data/obj/images/*.jpg")
print(images_list)

['data/obj/images/20200121_075041_010_jpg.rf.2623fe97ec679967fe5f5b281b5ede67.jpg', 'data/obj/images/20200119_142155_014_jpg.rf.46a86411ffe764feb96d707cc903a9b6.jpg', 'data/obj/images/20200119_133232_001_jpg.rf.d0dd373be36428b70c28c6594a44a9d5.jpg', 'data/obj/images/20200121_074837_006_jpg.rf.1c4d79f806ab734c08ab9e412e3c6bb8.jpg', 'data/obj/images/20200121_081500_009_jpg.rf.5044847633979083ccf16752dfb25753.jpg', 'data/obj/images/20200119_135225_020_jpg.rf.257a06ac6c819927151c5f63715b19a9.jpg', 'data/obj/images/20200119_183120_009_jpg.rf.33060b669e7d8e3b60aee726e512ff04.jpg', 'data/obj/images/20200120_175731_010_jpg.rf.9aa4edefe018b1cb9fb2b93712973018.jpg', 'data/obj/images/20200121_081450_002_jpg.rf.ef05d1d4c193d88a5460dd1cc2a18d85.jpg', 'data/obj/images/20200119_141057_001_jpg.rf.a690d8fbb7ce07319b17ebc369fc9c3c.jpg', 'data/obj/images/20200119_130613_013_jpg.rf.535f1591da74d30f0fa7e2a64cb31675.jpg', 'data/obj/images/20200120_174225_014_jpg.rf.43bab615c961fde7e3e70f6d63c97af6.jpg', 'da

In [10]:
#Create training.txt file
file = open("/content/darknet/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [11]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_r.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 1449254, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.902271), count: 1, class_loss = 0.000000, iou_loss = 0.006832, total_loss = 0.006832 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.799976), count: 2, class_loss = 0.000007, iou_loss = 0.054740, total_loss = 0.054747 
 total_bbox = 1449257, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.799363), count: 1, class_loss = 0.000000, iou_loss = 0.034770, total_loss = 0.034770 
v3 (